 # 1. DEFINE I/O DATA FRAMES

 ## Specifications can be found @
 > http://www.lsi.upc.edu/~srlconll/soft.html#srlconll

In [79]:
import numpy as np 
import pandas as pd 
import re 



MODEL_NAME='dblstm_crf'
HPARAMS_STR='lr5.00e-04_hs256x64_ctx-p1_glove_s50'
MODEL_V= '03'
INPUT_DIR= '../datasets/csvs/'
OUTPUT_DIR='../outputs/'

INPUT_PATH = '{:}{:}'.format(INPUT_DIR, 'zhou_valid.csv')
OUTPUT_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'Yhat_valid.csv')
TARGET_EVALUATION_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'conll_evaluate_valid.txt')
TARGET_GOLDENSTD_PATH=  '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'conll_golden_valid.txt')



print(INPUT_PATH)
print(OUTPUT_PATH)

../datasets/csvs/zhou_valid.csv
../outputs/dblstm_crf/lr5.00e-04_hs256x64_ctx-p1_glove_s50/03/Yhat_valid.csv


In [80]:
df_input= pd.read_csv(INPUT_PATH, index_col=0)
df_output=pd.read_csv(OUTPUT_PATH, index_col=0)
print(df_input.shape, df_output.shape)

(12298, 17) (12298, 2)


In [81]:
df= df_input.join(df_output, how='left')
print(df.shape)

(12298, 19)


In [77]:
df.head()

,ID,S,P,P_S,FORM,LEMMA,PRED,FUNC,M_R,CTX_P-3,CTX_P-2,CTX_P-1,CTX_P+1,CTX_P+2,CTX_P+3,ARG_0,ARG_1,Y_0,Y_1
IDX,,,,,,,,,,,,,,,,,,,
123846,1,2879,5100,0,Entre,entre,estar,-,0,entre,o,convidado,o,-,rei,(A2*,A2,(A2*,A2
123847,2,2879,5100,0,os,o,estar,-,0,entre,o,convidado,o,-,rei,*,A2,*,A2
123848,3,2879,5100,0,convidados,convidado,estar,-,0,entre,o,convidado,o,-,rei,*),A2,*),A2
123849,4,2879,5100,0,estão,estar,estar,estar,1,entre,o,convidado,o,-,rei,(V*),V,(V*),V
123850,5,2879,5100,0,o,o,estar,-,1,entre,o,convidado,o,-,rei,(A1*,A1,(A1*,A1


 # 2. Outputs Conll

In [82]:
def outputs_conll(df, target_column='Y_0'):
	'''
		Converts a dataset to conll format 
	'''
	df= df[['S', 'P', 'FUNC', target_column]]

	#Replace '-' making it easier to concatenate
	sub_fn= lambda x: re.sub('-', '', x)
	df['FUNC'] = df['FUNC'].apply(sub_fn)
    
	s0= min(df['S'])
	sn= max(df['S'])
	for i,si in enumerate(range(s0,sn+1)):
	    dfsi=df.loc[df['S'] == si,:]
	    p0= min(dfsi['P'])
	    pn= max(dfsi['P'])    
	    for j,p in enumerate(range(p0,pn+1)): # concatenate by argument columns
	        dfpj=dfsi.loc[df['P']==p,:].reset_index(drop=True)
	        if j==0:
	            dfp=dfpj[['FUNC',target_column]]
	            dfp=dfp.rename(columns={target_column: 'ARG0'})
	        else:
	            dfp['FUNC']=dfp['FUNC'].map(str).values + dfpj['FUNC'].map(str).values
	            dfp= pd.concat((dfp, dfpj[target_column]), axis=1)
	            dfp=dfp.rename(columns={target_column: 'ARG{:d}'.format(p-p0)})            
	    if i==0:        
	        dfconll= dfp
	    else:
	        dfconll= dfconll.append(dfp)
	    blank_series= pd.Series([None]*dfconll.shape[1], index=dfconll.columns)    
	    dfconll= dfconll.append(blank_series, ignore_index=True)
	        
	#Replace '-' making it easier to concatenate
	sub_fn= lambda x: '-' if isinstance(x,str) and len(x) == 0 else x
	dfconll['FUNC'] = dfconll['FUNC'].apply(sub_fn)
    
	#Replace Nans ( from skipping lines to empty)
	dfconll= dfconll.fillna('')
    
	#Order columns to fit conll standard
	num_columns=len(dfconll.columns)
	usecolumns=['FUNC'] + ['ARG{:d}'.format(i) for i in range(num_columns-1)]
	return dfconll[usecolumns]

 # 3. Make Evaluation

In [83]:

dfconll = outputs_conll(df)
dfconll.to_csv(TARGET_EVALUATION_PATH, sep= '\t', index=False, header=False)
dfconll.head()

/Users/Varela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,FUNC,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5
0,-,(A1*,,,,,
1,-,*,,,,,
2,-,*),,,,,
3,estar,(V*),,,,,
4,-,(A1*,,,,,


 # 4. Make Gold

In [70]:
dfconll_gold = outputs_conll(df, target_column='ARG_0')
dfconll_gold.to_csv(TARGET_GOLDENSTD_PATH, sep= '\t', index=False, header=False)
dfconll_gold.head()

/Users/Varela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,FUNC,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5
0,-,(A2*,,,,,
1,-,*,,,,,
2,-,*),,,,,
3,estar,(V*),,,,,
4,-,(A1*,,,,,
